build nn with Tensorflow

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from matplotlib import pyplot
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from keras.layers.core import Dense, Activation
from keras.utils.np_utils import to_categorical

In [2]:
data = pd.read_csv("./data/data.csv")
feature = data.columns.values 
feature = np.delete(feature,0)

y = data.iloc[:,0].values
x = data.drop(columns = ['discomfort']).values
x4= data[['ev_mean','omega_mode','pos_mean','ls_mean']].values
for i in range(len(y)):
    if y[i] == 'No':
        y[i] = 0
    else:
        y[i] = 1

deal np.nan

In [3]:
for i in range(len(x)):
    for j in range(len(x[i])):
        if np.isnan(x[i][j]):
            x[i][j] = 0.0001

In [4]:
for i in x:
    for j in i:
        if np.isnan(j):
            print('nan')
            
for i in y:
    if np.isnan(i):
            print('nan')

Normalization

In [5]:
for i in range(len(x)):
    for j in range(len(x[i])):
        if x[i][j] == 0.0:
            x[i][j] += 0.0001
    
scaler = StandardScaler()

x_scaled = scaler.fit_transform(
    x.astype(np.float32).reshape(-1,1))

for i in x:
    for j in i:
        if np.isnan(j):
            print('nan')
            
for i in y:
    if np.isnan(i):
            print('nan')

build model

In [6]:
model = Sequential()

nfeature = x.shape[1]

model.add(Dense(32, input_dim=nfeature))

model.add(Activation('relu'))

model.add(Dense(32, activation='softmax'))

model.add(Dense(1))

model.add(Activation('relu'))

opt = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss='mean_squared_error', optimizer=opt, metrics = ['accuracy'])


In [7]:
x_train = np.asarray(x[:200]).astype(np.float32)
x_test = np.asarray(x[200:]).astype(np.float32)
y_train = np.asarray(y[:200]).astype(np.float32)
y_test = np.asarray(y[200:]).astype(np.float32)

model.fit(x_train, y_train, batch_size=128, epochs=12, validation_data=(x_test, y_test), verbose=2)

Epoch 1/12
2/2 - 1s - loss: 0.4090 - accuracy: 0.5900 - val_loss: 0.4623 - val_accuracy: 0.5385 - 549ms/epoch - 274ms/step
Epoch 2/12
2/2 - 0s - loss: 0.4089 - accuracy: 0.5900 - val_loss: 0.4626 - val_accuracy: 0.5385 - 32ms/epoch - 16ms/step
Epoch 3/12
2/2 - 0s - loss: 0.4087 - accuracy: 0.5900 - val_loss: 0.4628 - val_accuracy: 0.5385 - 27ms/epoch - 13ms/step
Epoch 4/12
2/2 - 0s - loss: 0.4079 - accuracy: 0.5900 - val_loss: 0.4514 - val_accuracy: 0.5385 - 25ms/epoch - 13ms/step
Epoch 5/12
2/2 - 0s - loss: 0.3942 - accuracy: 0.5900 - val_loss: 0.4270 - val_accuracy: 0.5385 - 23ms/epoch - 12ms/step
Epoch 6/12
2/2 - 0s - loss: 0.3723 - accuracy: 0.5900 - val_loss: 0.3998 - val_accuracy: 0.5385 - 27ms/epoch - 13ms/step
Epoch 7/12
2/2 - 0s - loss: 0.3145 - accuracy: 0.5900 - val_loss: 0.2798 - val_accuracy: 0.5385 - 24ms/epoch - 12ms/step
Epoch 8/12
2/2 - 0s - loss: 0.2567 - accuracy: 0.5900 - val_loss: 0.2697 - val_accuracy: 0.5385 - 24ms/epoch - 12ms/step
Epoch 9/12
2/2 - 0s - loss: 0.

use dgp to predict

In [8]:
dgpcsv = pd.read_csv("./data/dgp.csv")
dgpcsv = dgpcsv[['Index','discomfort','Scene']].values
print(dgpcsv[:10]) 

situation = ['1-panel_low','1-panel_high','2-panel_low','2-panel_high']
num_situation = [0,1,2,3]
x_situation = [
                [1,0,0,0],
                [0,1,0,0],
                [0,0,1,0],
                [0,0,0,1]
]

dgplist=[]
no_sum = [0]*4
sumlist = [0]*4

for i in range(252):
    sit = dgpcsv[i][2]
    for j in range(4):
        if sit == situation [j]:
            sumlist[j]+=1
            if dgpcsv[i][1]=='No':
                no_sum[j]+=1
            elif dgpcsv[i][1]!='Yes':
                print('number: ',i,' error not y/n')
for i in no_sum:
    dgplist.append(i/63)
print(dgplist)

dgp_img=[]
for i in range(252):
    sit = dgpcsv[i][2]
    for j in range(4):
        if sit == situation [j]:
            dgp_img.append(dgplist[j])
    
            
print(len(dgp_img),dgp_img[:5])

y = dgp_img

[[1 'No' '1-panel_low']
 [1 'No' '2-panel_low']
 [1 'No' '2-panel_high']
 [1 'Yes' '1-panel_high']
 [2 'No' '1-panel_low']
 [2 'No' '1-panel_high']
 [2 'No' '2-panel_low']
 [2 'Yes' '2-panel_high']
 [3 'Yes' '2-panel_low']
 [3 'Yes' '1-panel_low']]
[0.5396825396825397, 0.5873015873015873, 0.6349206349206349, 0.5555555555555556]
252 [0.5396825396825397, 0.6349206349206349, 0.5555555555555556, 0.5873015873015873, 0.5396825396825397]


In [9]:
model = Sequential()

nfeature = x.shape[1]

model.add(Dense(32, input_dim=nfeature))

model.add(Activation('relu'))

model.add(Dense(32, activation='softmax'))

model.add(Dense(1))

model.add(Activation('relu'))

opt = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss='mean_squared_error', optimizer=opt)

In [10]:
x_train = np.asarray(x[:200]).astype(np.float32)
x_test = np.asarray(x[200:]).astype(np.float32)
y_train = np.asarray(y[:200]).astype(np.float32)
y_test = np.asarray(y[200:]).astype(np.float32)

model.fit(x_train, y_train, batch_size=128, epochs=12, validation_data=(x_test, y_test), verbose=2)

Epoch 1/12
2/2 - 0s - loss: 0.3089 - val_loss: 0.2466 - 391ms/epoch - 196ms/step
Epoch 2/12
2/2 - 0s - loss: 0.2405 - val_loss: 0.2124 - 36ms/epoch - 18ms/step
Epoch 3/12
2/2 - 0s - loss: 0.2063 - val_loss: 0.1790 - 25ms/epoch - 13ms/step
Epoch 4/12
2/2 - 0s - loss: 0.1733 - val_loss: 0.1480 - 23ms/epoch - 11ms/step
Epoch 5/12
2/2 - 0s - loss: 0.1428 - val_loss: 0.1200 - 26ms/epoch - 13ms/step
Epoch 6/12
2/2 - 0s - loss: 0.1155 - val_loss: 0.0953 - 23ms/epoch - 12ms/step
Epoch 7/12
2/2 - 0s - loss: 0.0913 - val_loss: 0.0739 - 25ms/epoch - 13ms/step
Epoch 8/12
2/2 - 0s - loss: 0.0705 - val_loss: 0.0557 - 31ms/epoch - 15ms/step
Epoch 9/12
2/2 - 0s - loss: 0.0530 - val_loss: 0.0407 - 25ms/epoch - 13ms/step
Epoch 10/12
2/2 - 0s - loss: 0.0384 - val_loss: 0.0287 - 23ms/epoch - 12ms/step
Epoch 11/12
2/2 - 0s - loss: 0.0269 - val_loss: 0.0193 - 25ms/epoch - 12ms/step
Epoch 12/12
2/2 - 0s - loss: 0.0180 - val_loss: 0.0124 - 24ms/epoch - 12ms/step


Update data

In [11]:
features_sort = ['Glare_diagram','ugr','dgi','log_gc','ugr_exp','cgi','participant','edir','ev','eindir',
                 'av_lum','med_lum','dgp','av_lum_cie','dgi_mod','Experiment','ugp',]


y_labels = ['Noticeable','Imperceptible','Disturbing','Intolerable']
co_label = [0,0,1,1]

data = pd.read_csv("./data/data2.csv")
data.dropna(axis = 1, how = 'all')
data.dropna(axis = 0, how = 'any')

feature = data.columns.values 
feature = np.delete(feature,0)
feature = np.delete(feature,2)

y = data.loc[:,['discomfort','describe']].values
x = data.drop(columns = ['discomfort','describe']).values

x_list =[]
flist =[]
for i in features_sort:
    flist.append(i)
    x_list.append(data.loc[:,flist].values)

new_y =[]
y_true =[]
num_yes = 0
for i in range(len(y)):
    js = 0
    if y[i][0] =='Yes':
        js = 4
        num_yes +=1
        y_true.append(1)
    else:
        y_true.append(0)
    for j in range(4):
        if y[i][1] == y_labels[j]:
            new_y.append(js +j)

print('yes_rate' , num_yes / x.shape[0]*100)
categorical_labels = to_categorical(new_y, num_classes=None)


for i in range(len(x)):
    if x[i][1] == '2A':
        x[i][1] = 2
    elif x[i][1] == '2B':
        x[i][1] = 3

for x in x_list: 
    for i in range(len(x)):
        for j in range(len(x[i])):
            if x[i][j] == '2A':
                x[i][j] = 2
            elif x[i][j] == '2B':
                x[i][j] = 3     


y = categorical_labels

x = np.asarray(x).astype(np.float32)
y = np.asarray(y).astype(np.float32)
print(x.shape,y.shape)

yes_rate 45.32710280373832
(428, 17) (428, 8)


In [12]:
for i in range(len(x)):
    for j in range(len(x[i])):
        if np.isnan(x[i][j]):
            x[i][j] = 0.0001

In [13]:
model = Sequential()

nfeature = x.shape[1]

model.add(Dense(32, input_dim=nfeature))

model.add(Activation('relu'))

model.add(Dense(32, activation='softmax'))

model.add(Dense(8))

model.add(Activation('softmax'))

opt = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss='mean_squared_error', optimizer='Adam', metrics = ['accuracy'])

In [14]:
x_train = np.asarray(x[:400]).astype(np.float32)
x_test = np.asarray(x[400:]).astype(np.float32)
y_train = np.asarray(y[:400]).astype(np.float32)
y_test = np.asarray(y[400:]).astype(np.float32)

model.fit(x_train, y_train, batch_size=8, epochs=100, validation_data=(x_test, y_test), verbose=2)

Epoch 1/100
50/50 - 0s - loss: 0.1073 - accuracy: 0.2725 - val_loss: 0.1060 - val_accuracy: 0.0714 - 450ms/epoch - 9ms/step
Epoch 2/100
50/50 - 0s - loss: 0.1053 - accuracy: 0.2400 - val_loss: 0.1043 - val_accuracy: 0.0714 - 67ms/epoch - 1ms/step
Epoch 3/100
50/50 - 0s - loss: 0.1037 - accuracy: 0.2525 - val_loss: 0.1042 - val_accuracy: 0.0714 - 77ms/epoch - 2ms/step
Epoch 4/100
50/50 - 0s - loss: 0.1025 - accuracy: 0.2500 - val_loss: 0.1024 - val_accuracy: 0.1071 - 71ms/epoch - 1ms/step
Epoch 5/100
50/50 - 0s - loss: 0.1013 - accuracy: 0.2500 - val_loss: 0.1014 - val_accuracy: 0.1786 - 72ms/epoch - 1ms/step
Epoch 6/100
50/50 - 0s - loss: 0.1003 - accuracy: 0.2550 - val_loss: 0.1003 - val_accuracy: 0.2143 - 74ms/epoch - 1ms/step
Epoch 7/100
50/50 - 0s - loss: 0.0996 - accuracy: 0.2700 - val_loss: 0.0995 - val_accuracy: 0.1786 - 56ms/epoch - 1ms/step
Epoch 8/100
50/50 - 0s - loss: 0.0989 - accuracy: 0.2700 - val_loss: 0.0991 - val_accuracy: 0.1786 - 86ms/epoch - 2ms/step
Epoch 9/100
50/

50/50 - 0s - loss: 0.0946 - accuracy: 0.2950 - val_loss: 0.0944 - val_accuracy: 0.2500 - 75ms/epoch - 2ms/step
Epoch 68/100
50/50 - 0s - loss: 0.0946 - accuracy: 0.2950 - val_loss: 0.0945 - val_accuracy: 0.2500 - 78ms/epoch - 2ms/step
Epoch 69/100
50/50 - 0s - loss: 0.0946 - accuracy: 0.2950 - val_loss: 0.0944 - val_accuracy: 0.2500 - 71ms/epoch - 1ms/step
Epoch 70/100
50/50 - 0s - loss: 0.0946 - accuracy: 0.2950 - val_loss: 0.0944 - val_accuracy: 0.2500 - 58ms/epoch - 1ms/step
Epoch 71/100
50/50 - 0s - loss: 0.0946 - accuracy: 0.2950 - val_loss: 0.0944 - val_accuracy: 0.2500 - 86ms/epoch - 2ms/step
Epoch 72/100
50/50 - 0s - loss: 0.0946 - accuracy: 0.2950 - val_loss: 0.0944 - val_accuracy: 0.2500 - 69ms/epoch - 1ms/step
Epoch 73/100
50/50 - 0s - loss: 0.0946 - accuracy: 0.2950 - val_loss: 0.0943 - val_accuracy: 0.2500 - 58ms/epoch - 1ms/step
Epoch 74/100
50/50 - 0s - loss: 0.0946 - accuracy: 0.2950 - val_loss: 0.0944 - val_accuracy: 0.2500 - 74ms/epoch - 1ms/step
Epoch 75/100
50/50 - 

In [15]:
y_pred = model.predict(x)
pred = []
for i in y_pred:
    p=0
    for j in range(4):
        p +=i[j]
    if p>0.5:
        pred.append(0)
    else:
        pred.append(1)

accuracy_score(y_true,pred)

0.5467289719626168

In [16]:
ac_list = []

In [17]:
for x in x_list[:5]:
    
    print('number of features : ',x.shape[1])
    x_train = np.asarray(x[:400]).astype(np.float32)
    x_test = np.asarray(x[400:]).astype(np.float32)
    y_train = np.asarray(y[:400]).astype(np.float32)
    y_test = np.asarray(y[400:]).astype(np.float32)
    model = Sequential()

    nfeature = x.shape[1]

    model.add(Dense(32, input_dim=nfeature))

    model.add(Activation('relu'))

    model.add(Dense(32, activation='softmax'))

    model.add(Dense(8))

    model.add(Activation('softmax'))

    opt = keras.optimizers.Adam(learning_rate=0.01)

    model.compile(loss='mean_squared_error', optimizer='Adam', metrics = ['accuracy'])

    model.fit(x_train, y_train, batch_size=8, epochs=20, validation_data=(x_test, y_test))

    y_pred = model.predict(x)
    pred = []
    for i in y_pred:
        p=0
        for j in range(4):
            p +=i[j]
        if p>0.5:
            pred.append(0)
        else:
            pred.append(1)
    score = accuracy_score(y_true,pred)
    ac_list.append(score)
    print('accuracy : ',score)

number of features :  1
Epoch 1/20
50/50 [==============================] - 0s 3ms/step - loss: 0.1083 - accuracy: 0.2275 - val_loss: 0.1067 - val_accuracy: 0.2857
Epoch 2/20
50/50 [==============================] - 0s 2ms/step - loss: 0.1061 - accuracy: 0.2775 - val_loss: 0.1048 - val_accuracy: 0.2857
Epoch 3/20
50/50 [==============================] - 0s 1ms/step - loss: 0.1040 - accuracy: 0.2775 - val_loss: 0.1031 - val_accuracy: 0.2857
Epoch 4/20
50/50 [==============================] - 0s 1ms/step - loss: 0.1020 - accuracy: 0.2775 - val_loss: 0.1017 - val_accuracy: 0.2857
Epoch 5/20
50/50 [==============================] - 0s 1ms/step - loss: 0.1001 - accuracy: 0.2725 - val_loss: 0.1004 - val_accuracy: 0.1786
Epoch 6/20
50/50 [==============================] - 0s 1ms/step - loss: 0.0982 - accuracy: 0.3425 - val_loss: 0.0991 - val_accuracy: 0.1786
Epoch 7/20
50/50 [==============================] - 0s 2ms/step - loss: 0.0963 - accuracy: 0.3425 - val_loss: 0.0983 - val_accuracy: 0.1

50/50 [==============================] - 0s 1ms/step - loss: 0.0827 - accuracy: 0.4900 - val_loss: 0.0924 - val_accuracy: 0.2143
Epoch 19/20
50/50 [==============================] - 0s 1ms/step - loss: 0.0820 - accuracy: 0.4900 - val_loss: 0.0923 - val_accuracy: 0.2143
Epoch 20/20
50/50 [==============================] - 0s 1ms/step - loss: 0.0814 - accuracy: 0.4900 - val_loss: 0.0920 - val_accuracy: 0.2143
accuracy :  0.7873831775700935
number of features :  4
Epoch 1/20
50/50 [==============================] - 0s 3ms/step - loss: 0.1071 - accuracy: 0.1800 - val_loss: 0.1051 - val_accuracy: 0.3929
Epoch 2/20
50/50 [==============================] - 0s 1ms/step - loss: 0.1038 - accuracy: 0.2425 - val_loss: 0.1046 - val_accuracy: 0.0714
Epoch 3/20
50/50 [==============================] - 0s 2ms/step - loss: 0.1017 - accuracy: 0.2475 - val_loss: 0.1033 - val_accuracy: 0.0714
Epoch 4/20
50/50 [==============================] - 0s 1ms/step - loss: 0.1006 - accuracy: 0.2475 - val_loss: 0.10

In [18]:
for x in x_list[5:10]:
    
    print('number of features : ',x.shape[1])
    x_train = np.asarray(x[:400]).astype(np.float32)
    x_test = np.asarray(x[400:]).astype(np.float32)
    y_train = np.asarray(y[:400]).astype(np.float32)
    y_test = np.asarray(y[400:]).astype(np.float32)
    model = Sequential()

    nfeature = x.shape[1]

    model.add(Dense(32, input_dim=nfeature))

    model.add(Activation('relu'))

    model.add(Dense(32, activation='softmax'))

    model.add(Dense(8))

    model.add(Activation('softmax'))

    opt = keras.optimizers.Adam(learning_rate=0.01)

    model.compile(loss='mean_squared_error', optimizer='Adam', metrics = ['accuracy'])

    model.fit(x_train, y_train, batch_size=8, epochs=20, validation_data=(x_test, y_test))

    y_pred = model.predict(x)
    pred = []
    for i in y_pred:
        p=0
        for j in range(4):
            p +=i[j]
        if p>0.5:
            pred.append(0)
        else:
            pred.append(1)

    score = accuracy_score(y_true,pred)
    ac_list.append(score)
    print('accuracy : ',score)

number of features :  6
Epoch 1/20
50/50 [==============================] - 1s 5ms/step - loss: 0.1096 - accuracy: 0.1175 - val_loss: 0.1062 - val_accuracy: 0.2500
Epoch 2/20
50/50 [==============================] - 0s 2ms/step - loss: 0.1060 - accuracy: 0.2225 - val_loss: 0.1037 - val_accuracy: 0.2500
Epoch 3/20
50/50 [==============================] - 0s 2ms/step - loss: 0.1027 - accuracy: 0.2825 - val_loss: 0.1011 - val_accuracy: 0.2857
Epoch 4/20
50/50 [==============================] - 0s 1ms/step - loss: 0.1011 - accuracy: 0.2825 - val_loss: 0.1001 - val_accuracy: 0.2857
Epoch 5/20
50/50 [==============================] - 0s 2ms/step - loss: 0.1000 - accuracy: 0.2875 - val_loss: 0.0994 - val_accuracy: 0.2857
Epoch 6/20
50/50 [==============================] - 0s 2ms/step - loss: 0.0992 - accuracy: 0.2875 - val_loss: 0.0989 - val_accuracy: 0.2857
Epoch 7/20
50/50 [==============================] - 0s 1ms/step - loss: 0.0986 - accuracy: 0.2875 - val_loss: 0.0986 - val_accuracy: 0.2

50/50 [==============================] - 0s 2ms/step - loss: 0.0963 - accuracy: 0.2675 - val_loss: 0.0975 - val_accuracy: 0.2857
Epoch 19/20
50/50 [==============================] - 0s 1ms/step - loss: 0.0962 - accuracy: 0.2825 - val_loss: 0.0976 - val_accuracy: 0.2857
Epoch 20/20
50/50 [==============================] - 0s 1ms/step - loss: 0.0961 - accuracy: 0.2775 - val_loss: 0.0976 - val_accuracy: 0.2857
accuracy :  0.5467289719626168
number of features :  9
Epoch 1/20
50/50 [==============================] - 0s 4ms/step - loss: 0.1062 - accuracy: 0.2475 - val_loss: 0.1076 - val_accuracy: 0.0714
Epoch 2/20
50/50 [==============================] - 0s 1ms/step - loss: 0.1047 - accuracy: 0.2500 - val_loss: 0.1064 - val_accuracy: 0.0714
Epoch 3/20
50/50 [==============================] - 0s 2ms/step - loss: 0.1034 - accuracy: 0.2500 - val_loss: 0.1052 - val_accuracy: 0.0714
Epoch 4/20
50/50 [==============================] - 0s 2ms/step - loss: 0.1023 - accuracy: 0.2500 - val_loss: 0.10

In [19]:
for x in x_list[10:15]:
    
    print('number of features : ',x.shape[1])
    x_train = np.asarray(x[:400]).astype(np.float32)
    x_test = np.asarray(x[400:]).astype(np.float32)
    y_train = np.asarray(y[:400]).astype(np.float32)
    y_test = np.asarray(y[400:]).astype(np.float32)
    model = Sequential()

    nfeature = x.shape[1]

    model.add(Dense(32, input_dim=nfeature))

    model.add(Activation('relu'))

    model.add(Dense(32, activation='softmax'))

    model.add(Dense(8))

    model.add(Activation('softmax'))

    opt = keras.optimizers.Adam(learning_rate=0.01)

    model.compile(loss='mean_squared_error', optimizer=opt, metrics = ['accuracy'])

    model.fit(x_train, y_train, batch_size=8, epochs=20, validation_data=(x_test, y_test))

    y_pred = model.predict(x)
    pred = []
    for i in y_pred:
        p=0
        for j in range(4):
            p +=i[j]
        if p>0.5:
            pred.append(0)
        else:
            pred.append(1)

    score = accuracy_score(y_true,pred)
    ac_list.append(score)
    print('accuracy : ',score)

number of features :  11
Epoch 1/20
50/50 [==============================] - 0s 4ms/step - loss: 0.1036 - accuracy: 0.2625 - val_loss: 0.1014 - val_accuracy: 0.0714
Epoch 2/20
50/50 [==============================] - 0s 1ms/step - loss: 0.0974 - accuracy: 0.2175 - val_loss: 0.0979 - val_accuracy: 0.2857
Epoch 3/20
50/50 [==============================] - 0s 1ms/step - loss: 0.0960 - accuracy: 0.2775 - val_loss: 0.0976 - val_accuracy: 0.2857
Epoch 4/20
50/50 [==============================] - 0s 2ms/step - loss: 0.0957 - accuracy: 0.2800 - val_loss: 0.0982 - val_accuracy: 0.2857
Epoch 5/20
50/50 [==============================] - 0s 3ms/step - loss: 0.0955 - accuracy: 0.2650 - val_loss: 0.0970 - val_accuracy: 0.2500
Epoch 6/20
50/50 [==============================] - 0s 2ms/step - loss: 0.0955 - accuracy: 0.2575 - val_loss: 0.0971 - val_accuracy: 0.2857
Epoch 7/20
50/50 [==============================] - 0s 1ms/step - loss: 0.0953 - accuracy: 0.2800 - val_loss: 0.0967 - val_accuracy: 0.

50/50 [==============================] - 0s 1ms/step - loss: 0.0954 - accuracy: 0.2775 - val_loss: 0.0968 - val_accuracy: 0.2857
Epoch 19/20
50/50 [==============================] - 0s 2ms/step - loss: 0.0954 - accuracy: 0.2525 - val_loss: 0.0974 - val_accuracy: 0.2857
Epoch 20/20
50/50 [==============================] - 0s 1ms/step - loss: 0.0954 - accuracy: 0.2775 - val_loss: 0.0964 - val_accuracy: 0.2857
accuracy :  0.5467289719626168
number of features :  14
Epoch 1/20
50/50 [==============================] - 0s 4ms/step - loss: 0.1043 - accuracy: 0.2150 - val_loss: 0.0984 - val_accuracy: 0.2500
Epoch 2/20
50/50 [==============================] - 0s 1ms/step - loss: 0.0970 - accuracy: 0.2550 - val_loss: 0.0978 - val_accuracy: 0.2857
Epoch 3/20
50/50 [==============================] - 0s 1ms/step - loss: 0.0960 - accuracy: 0.2475 - val_loss: 0.0982 - val_accuracy: 0.2857
Epoch 4/20
50/50 [==============================] - 0s 1ms/step - loss: 0.0958 - accuracy: 0.2175 - val_loss: 0.0